# Table values

Storing the function calls for calculating the values in tables 1 and 2.

In [1]:
import sys
# Add common resources folder to path
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/Common/")
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png
    )

from classes import SatComp_Metric, CT_SLF_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Check running location and adjust working directory appropriately.

In [2]:
host = os.uname()[1]
if 'jupyter' in host.split('-'): # Check if running on NIRD through the Jupyter Hub
    print('Running through MC2 Jupyter Hub')
    model_dir = '/mnt/mcc-ns9600k/jonahks/'
    os.chdir(model_dir)

else:  # Assume that we're running on a local machine and mounting NIRD
    print('Running on %s, attempting to mount ns9600k/jonahks/ from NIRD' % str(host))
    os.system('fusermount -zu ~/drivemount/')  # unmount first
    os.system('sshfs jonahks@login.nird.sigma2.no:"p/jonahks/" ~/drivemount/')    # Calling mountnird from .bashrc doesn't work
    os.chdir('/home/jonahks/drivemount/')
    save_dir = '~/DATAOUT/'
    save_to = os.path.expanduser(save_dir)

output_dir = 'figures/'
case_dir = 'satcomp/'   # inconsistent label compared to jupy_test
conv_dir ='convectivephase/'

# Check that each important directory can be accessed:    
access_paths = os.path.exists(output_dir) and os.path.exists(case_dir) and os.path.exists(conv_dir)
print('Can access all directory paths:', access_paths)

Running through MC2 Jupyter Hub
Can access all directory paths: True


In [ ]:
allmetric = SatComp_Metric(case_dir)

allmetric.add_case('20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1', label="CAM6-Oslo")
allmetric.add_case('CESM2_slfvars', label="CAM6")
allmetric.add_case('20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1', label="CAM6-OsloIce")

allmetric.add_case("20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10",label='CAM6-Oslo \n Fit 1')
allmetric.add_case('20200629_morn_cam61satcomp_wbf_0.5_inp_0.05',label='CAM6-OsloIce \n Fit 2')
allmetric.add_case('20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1',label='CAM6-OsloIce \n Fit 3')
allmetric.add_case('20200713_CESM2_satcomp_wbf_1_inp_100',label="CAM6 Fit")

In [3]:
completemetric = SatComp_Metric(case_dir) # Add completeness runs.

completemetric.add_case('20211124_102800_cam6satcomp16_wbf_1.25_inp_1',label="CAM6 WBF:1.25 INP:1")
completemetric.add_case('20211124_102800_cam6satcomp16_wbf_1_inp_10',label="CAM6 WBF:1 INP:10")
completemetric.add_case('20211124_103400_cam6satcomp16_wbf_0.5_inp_1',label="CAM61 WBF:0.5 INP:1")
completemetric.add_case('20211124_103600_cam6satcomp16_wbf_1_inp_0.05',label="CAM61 WBF:1 INP:0.05")
completemetric.add_case('20211124_103800_cam6satcomp16_wbf_0.2_inp_1',label="CAM61 WBF:0.2 INP:1")
completemetric.add_case('20211124_103900_cam6satcomp16_wbf_1_inp_0.1',label="CAM61 WBF:1 INP:0.1")

Loading GOCCP data...done.
Loading CALIOP SLFs...done
Loading CERES-EBAF fluxes...done.
Trying to load concatenated file for 20211124_102800_cam6satcomp16_wbf_1.25_inp_1
20211124_102800_cam6satcomp16_wbf_1.25_inp_1 load successfully.
Trying to load concatenated file for 20211124_102800_cam6satcomp16_wbf_1_inp_10
20211124_102800_cam6satcomp16_wbf_1_inp_10 load successfully.
Trying to load concatenated file for 20211124_103400_cam6satcomp16_wbf_0.5_inp_1
20211124_103400_cam6satcomp16_wbf_0.5_inp_1 load successfully.
Trying to load concatenated file for 20211124_103600_cam6satcomp16_wbf_1_inp_0.05
20211124_103600_cam6satcomp16_wbf_1_inp_0.05 load successfully.
Trying to load concatenated file for 20211124_103800_cam6satcomp16_wbf_0.2_inp_1
20211124_103800_cam6satcomp16_wbf_0.2_inp_1 load successfully.
Trying to load concatenated file for 20211124_103900_cam6satcomp16_wbf_1_inp_0.1
20211124_103900_cam6satcomp16_wbf_1_inp_0.1 load successfully.


# Table 1

## Ice concentration at 860hPa averaged between 66-82N.

In [4]:
for i in allmetric.get_cases():
    _case = allmetric.get_case(i)
    _da = _case.case_da
    _da = _da.sel(lat=slice(66,82))
    _da = _da.sel(lev = 860, method='nearest') # 860
    
    _weights = _da['cell_weight'] # I could easily weight by month length here
    try:
        ice_rad = _da['AWNI'] / _da['FREQI']
        mean_rad = masked_average(ice_rad, dim=['lat','lon','time'], 
                                  weights=_weights)
#         mean_rad = ice_rad.mean(['time','lat','lon'])
        print(i,': ',mean_rad.values)
    except:
        pass
#     ice_rad = _case['AREI'] /

20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1 :  4116.331637568774
CESM2_slfvars :  5551.3886610670315
20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1 :  15666.312188603899
20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10 :  3867.513203404934
20200629_morn_cam61satcomp_wbf_0.5_inp_0.05 :  5412.88870268456
20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1 :  8604.530560860781
20200713_CESM2_satcomp_wbf_1_inp_100 :  5060.4165295560815


In [4]:
for i in completemetric.get_cases():
    _case = completemetric.get_case(i)
    _da = _case.case_da
    _da = _da.sel(lat=slice(66,82))
    _da = _da.sel(lev = 860, method='nearest') # 860
    
    _weights = _da['cell_weight']
    try:
        ice_rad = _da['AWNI'] / _da['FREQI']
        mean_rad = masked_average(ice_rad, dim=['lat','lon','time'], 
                                  weights=_weights)
#         mean_rad = ice_rad.mean(['time','lat','lon'])
        print(i,': ',mean_rad.values)
    except:
        pass
#     ice_rad = _case['AREI'] /

20211124_102800_cam6satcomp16_wbf_1.25_inp_1 :  3950.623673315101
20211124_102800_cam6satcomp16_wbf_1_inp_10 :  4015.6684108803734
20211124_103400_cam6satcomp16_wbf_0.5_inp_1 :  16538.926897767356
20211124_103600_cam6satcomp16_wbf_1_inp_0.05 :  4252.489757663627
20211124_103800_cam6satcomp16_wbf_0.2_inp_1 :  23462.9791559725
20211124_103900_cam6satcomp16_wbf_1_inp_0.1 :  4427.70474902864


## Ice size at 860hPa averaged between 66-82N.

In [15]:
for i in allmetric.get_cases():
    _case = allmetric.get_case(i)
    _da = _case.case_da
    _da = _da.sel(lat=slice(66,82))
    _da = _da.sel(lev = 860, method='nearest') # 860
    
    _weights = _da['cell_weight']
    try:
        ice_rad = _da['AREI'] / _da['FREQI']
        mean_rad = masked_average(ice_rad, dim=['lat','lon','time'], 
                                  weights=_weights)
#         mean_rad = ice_rad.mean(['time','lat','lon']) # old no lat weighting
        print(i,': ',mean_rad.values)
    except:
        pass
#     ice_rad = _case['AREI'] /

20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1 :  150.70281258639352
CESM2_slfvars :  164.52245121208912
20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1 :  132.43616186219836
20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10 :  163.31143531764374
20200629_morn_cam61satcomp_wbf_0.5_inp_0.05 :  124.17246029942044
20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1 :  112.3887552872834
20200713_CESM2_satcomp_wbf_1_inp_100 :  209.22911677769198


In [5]:
for i in completemetric.get_cases():
    _case = completemetric.get_case(i)
    _da = _case.case_da
    _da = _da.sel(lat=slice(66,82))
    _da = _da.sel(lev = 860, method='nearest') # 860
    
    _weights = _da['cell_weight']
    try:
        ice_rad = _da['AREI'] / _da['FREQI']
        mean_rad = masked_average(ice_rad, dim=['lat','lon','time'], 
                                  weights=_weights)
#         mean_rad = ice_rad.mean(['time','lat','lon']) # old no lat weighting
        print(i,': ',mean_rad.values)
    except:
        pass
#     ice_rad = _case['AREI'] /

20211124_102800_cam6satcomp16_wbf_1.25_inp_1 :  155.62724948577082
20211124_102800_cam6satcomp16_wbf_1_inp_10 :  160.0204175361136
20211124_103400_cam6satcomp16_wbf_0.5_inp_1 :  122.83047645996525
20211124_103600_cam6satcomp16_wbf_1_inp_0.05 :  134.0904181200037
20211124_103800_cam6satcomp16_wbf_0.2_inp_1 :  111.60986190475714
20211124_103900_cam6satcomp16_wbf_1_inp_0.1 :  134.07755958249388


# Table 2

### Total cloud fraction

In [22]:
tot_bias = allmetric.band_bias("CLDTOT_CAL",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.p

In [6]:
tot_bias_complete = completemetric.band_bias("CLDTOT_CAL",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


### Liquid phase cloud fraction

In [23]:
liq_bias = allmetric.band_bias("CLDTOT_CAL_LIQ",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.p

In [7]:
liq_bias_complete = completemetric.band_bias("CLDTOT_CAL_LIQ",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


### Ice phase cloud fraction

In [24]:
ice_bias = allmetric.band_bias("CLDTOT_CAL_ICE",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.p

In [8]:
ice_bias_complete = completemetric.band_bias("CLDTOT_CAL_ICE",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


### Un-phased cloud fraction

In [25]:
un_bias = allmetric.band_bias("CLDTOT_CAL_UN",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.p

In [9]:
un_bias_complete = completemetric.band_bias("CLDTOT_CAL_UN",[66,82])

/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


### Shortwave cloud feedback

In [17]:
allmetric.band_bias("SWCF",[66,82])

[['CAM6-Oslo', array(-3.5374576)],
 ['CAM6', array(-3.98126866)],
 ['CAM6-OsloIce', array(-2.92430609)],
 ['CAM6-Oslo \n Fit 1', array(-2.99536988)],
 ['CAM6-OsloIce \n Fit 2', array(-3.68558189)],
 ['CAM6-OsloIce \n Fit 3', array(-5.13628388)],
 ['CAM6 Fit', array(-3.31257517)]]

In [10]:
completemetric.band_bias("SWCF",[66,82])

[['CAM6 WBF:1.25 INP:1', array(-3.12971373)],
 ['CAM6 WBF:1 INP:10', array(-3.39652486)],
 ['CAM61 WBF:0.5 INP:1', array(-4.12552808)],
 ['CAM61 WBF:1 INP:0.05', array(-2.21374588)],
 ['CAM61 WBF:0.2 INP:1', array(-5.49149234)],
 ['CAM61 WBF:1 INP:0.1', array(-2.15498887)]]

### Longwave cloud feedback

In [16]:
allmetric.band_bias("LWCF",[66,82])

[['CAM6-Oslo', array(-0.07488647)],
 ['CAM6', array(-0.77955509)],
 ['CAM6-OsloIce', array(0.31772517)],
 ['CAM6-Oslo \n Fit 1', array(-0.61040279)],
 ['CAM6-OsloIce \n Fit 2', array(0.43195072)],
 ['CAM6-OsloIce \n Fit 3', array(1.82633313)],
 ['CAM6 Fit', array(-1.94298835)]]

In [11]:
completemetric.band_bias("LWCF",[66,82])

[['CAM6 WBF:1.25 INP:1', array(-0.44379185)],
 ['CAM6 WBF:1 INP:10', array(-0.27383858)],
 ['CAM61 WBF:0.5 INP:1', array(1.15739832)],
 ['CAM61 WBF:1 INP:0.05', array(-0.74146593)],
 ['CAM61 WBF:0.2 INP:1', array(2.46042415)],
 ['CAM61 WBF:1 INP:0.1', array(-0.78519756)]]

In [26]:
tot_bias

[['CAM6-Oslo', array(-2.0106763)],
 ['CAM6', array(2.13134918)],
 ['CAM6-OsloIce', array(-5.79585754)],
 ['CAM6-Oslo \n Fit 1', array(-3.55012017)],
 ['CAM6-OsloIce \n Fit 2', array(-2.00652738)],
 ['CAM6-OsloIce \n Fit 3', array(-0.33358325)],
 ['CAM6 Fit', array(-5.07441276)]]

In [12]:
tot_bias_complete

[['CAM6 WBF:1.25 INP:1', array(-2.81605473)],
 ['CAM6 WBF:1 INP:10', array(-2.7964009)],
 ['CAM61 WBF:0.5 INP:1', array(-4.15692974)],
 ['CAM61 WBF:1 INP:0.05', array(-4.297861)],
 ['CAM61 WBF:0.2 INP:1', array(-2.13839722)],
 ['CAM61 WBF:1 INP:0.1', array(-4.863211)]]

In [27]:
liq_bias

[['CAM6-Oslo', array(-0.6565369)],
 ['CAM6', array(11.21875298)],
 ['CAM6-OsloIce', array(-8.20726302)],
 ['CAM6-Oslo \n Fit 1', array(-2.18409939)],
 ['CAM6-OsloIce \n Fit 2', array(-0.94753746)],
 ['CAM6-OsloIce \n Fit 3', array(-1.26644176)],
 ['CAM6 Fit', array(5.36442114)]]

In [13]:
liq_bias_complete

[['CAM6 WBF:1.25 INP:1', array(-1.50226818)],
 ['CAM6 WBF:1 INP:10', array(-1.37811709)],
 ['CAM61 WBF:0.5 INP:1', array(-6.779906)],
 ['CAM61 WBF:1 INP:0.05', array(-3.32224463)],
 ['CAM61 WBF:0.2 INP:1', array(-6.44051318)],
 ['CAM61 WBF:1 INP:0.1', array(-4.26679845)]]

In [28]:
ice_bias

[['CAM6-Oslo', array(0.72205001)],
 ['CAM6', array(-7.91897894)],
 ['CAM6-OsloIce', array(6.12218441)],
 ['CAM6-Oslo \n Fit 1', array(1.12950528)],
 ['CAM6-OsloIce \n Fit 2', array(1.63510622)],
 ['CAM6-OsloIce \n Fit 3', array(3.57021379)],
 ['CAM6 Fit', array(-8.13947182)]]

In [14]:
ice_bias_complete

[['CAM6 WBF:1.25 INP:1', array(1.02336021)],
 ['CAM6 WBF:1 INP:10', array(0.84548857)],
 ['CAM61 WBF:0.5 INP:1', array(6.0703193)],
 ['CAM61 WBF:1 INP:0.05', array(2.11938443)],
 ['CAM61 WBF:0.2 INP:1', array(7.59677503)],
 ['CAM61 WBF:1 INP:0.1', array(2.63535043)]]

In [34]:
un_bias

[['CAM6-Oslo', array(-1.75538762)],
 ['CAM6', array(-0.84762339)],
 ['CAM6-OsloIce', array(-3.3899766)],
 ['CAM6-Oslo \n Fit 1', array(-2.17472385)],
 ['CAM6-OsloIce \n Fit 2', array(-2.37329421)],
 ['CAM6-OsloIce \n Fit 3', array(-2.31655364)],
 ['CAM6 Fit', array(-1.97855967)]]

In [15]:
un_bias_complete

[['CAM6 WBF:1.25 INP:1', array(-2.01634485)],
 ['CAM6 WBF:1 INP:10', array(-1.94297046)],
 ['CAM61 WBF:0.5 INP:1', array(-3.12654094)],
 ['CAM61 WBF:1 INP:0.05', array(-2.77419849)],
 ['CAM61 WBF:0.2 INP:1', array(-2.97385713)],
 ['CAM61 WBF:1 INP:0.1', array(-2.91096073)]]

## Trying to understand why the CALIOP GOCCP biases by phase don't equal the total bias...

### Total cloud bias is not the sum of the cloud component biases. This seems inconsistent.

In [39]:
np.array(liq_bias)[:,1].astype('float') + np.array(ice_bias)[:,1].astype('float') + np.array(un_bias)[:,1].astype('float')

array([-1.68987451,  2.45215065, -5.47505521, -3.22931796, -1.68572544,
       -0.01278161, -4.75361036])

In [40]:
np.array(tot_bias)[:,1].astype('float')

array([-2.0106763 ,  2.13134918, -5.79585754, -3.55012017, -2.00652738,
       -0.33358325, -5.07441276])

### This comes from the GOCCP observations, not my models or analysis

In [52]:
goccp = add_weights(allmetric.goccp_data)
weights = goccp['cell_weight']

In [56]:
cldtot_avg = masked_average(goccp['CLDTOT_CAL'][0,:,:],dim=['lat','lon'],weights=weights)

cldliq_avg = masked_average(goccp['CLDTOT_CAL_LIQ'][0,:,:],dim=['lat','lon'],weights=weights)
cldice_avg = masked_average(goccp['CLDTOT_CAL_ICE'][0,:,:],dim=['lat','lon'],weights=weights)
cldun_avg = masked_average(goccp['CLDTOT_CAL_UN'][0,:,:],dim=['lat','lon'],weights=weights)

### The total cloud is slightly (0.35%) more than the sum of the phase components.

In [55]:
cldtot_avg.values

array(66.60783959)

In [60]:
cldliq_avg.values + cldice_avg.values + cldun_avg.values

66.25326536949268

### Is this inconsistency reproduced by COSP?

In [63]:
noresm = allmetric.get_case('20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1').case_da

In [69]:
noresm_da = add_weights(noresm)
weights2 = noresm_da['cell_weight']

In [70]:
cldtot_avg_mod = masked_average(noresm_da['CLDTOT_CAL'][0,:,:],dim=['lat','lon'],weights=weights2)

cldliq_avg_mod = masked_average(noresm_da['CLDTOT_CAL_LIQ'][0,:,:],dim=['lat','lon'],weights=weights2)
cldice_avg_mod = masked_average(noresm_da['CLDTOT_CAL_ICE'][0,:,:],dim=['lat','lon'],weights=weights2)
cldun_avg_mod = masked_average(noresm_da['CLDTOT_CAL_UN'][0,:,:],dim=['lat','lon'],weights=weights2)

In [71]:
cldtot_avg_mod.values

array(58.6553173)

In [72]:
cldliq_avg_mod.values + cldice_avg_mod.values + cldun_avg_mod.values

58.65531730403106

### No, it isn't!! Case closed!